## Import Libraries

In [1]:
import pandas as pd

The helper functions to read in the Zillow data are in a file called helpers.py in the 'scripts' directory in the root of our project.

In [2]:
# add the scripts directory to the python path, 
# or the set of places the python interpreter looks for code files
import sys
sys.path.insert(0, './scripts')

# now read in the functions
from helpers import read_in_dataset, merge_dataset

# look up the code in read_in_dataset()
#import inspect
#print(inspect.getsource(read_in_dataset))

## Read in the Data

In [3]:
prop_fname = 'data/unzipped_data/properties_2016.csv'
properties = read_in_dataset(prop_fname, verbose=True)

train_fname = 'data/unzipped_data/train_2016.csv'
train_raw = read_in_dataset(train_fname, verbose=True)

/anaconda3/envs/insight/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3214: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):



******** Reading in the data/unzipped_data/properties_2016.csv dataset *********

it has 2985217 rows and 58 columns

*********** Reading in the data/unzipped_data/train_2016.csv dataset ***********

it has 90275 rows and 3 columns


In [4]:
properties.head()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.0,2015.0,9.0,NaN,NaN,NaN,NaN
1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN
2,10843547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,650756.0,1413387.0,2015.0,762631.0,20800.37,NaN,NaN,NaN
3,10859147,NaN,NaN,NaN,0.0,0.0,3.0,7.0,NaN,NaN,...,1.0,NaN,571346.0,1156834.0,2015.0,585488.0,14557.57,NaN,NaN,NaN
4,10879947,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,NaN,...,NaN,NaN,193796.0,433491.0,2015.0,239695.0,5725.17,NaN,NaN,NaN


In [5]:
train_raw.head()

,parcelid,logerror,transactiondate
0,11016594,0.0276,2016-01-01
1,14366692,-0.1684,2016-01-01
2,12098116,-0.0040,2016-01-01
3,12643413,0.0218,2016-01-02
4,14432541,-0.0050,2016-01-02


## Explore the Raw Datasets

In [6]:
# how many train records don't have a matching property
len(set(train_raw.parcelid) - set(properties.parcelid))

0

In [7]:
# how many properties don't have a matching train record
len(set(properties.parcelid) - set(train_raw.parcelid))

2895067

So there are a lot of properties that don't have a corresponding target (logerror). What could we use these records for? For now, lets separate them out so we don't have to carry them around in memory.

## Create Dataset for Analysis
Lets create a file with only the properties and sales records we'll need for training.

In [8]:
# inspect the function we made for this
#print(inspect.getsource(merge_dataset))

# merge the data and remove the raw datasets
train_data_merged = merge_dataset(train_raw, properties)
del train_raw
del properties

In [9]:
train_data_merged.head()

,parcelid,logerror,transactiondate,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,11016594,0.0276,2016-01-01,1.0,NaN,NaN,2.0,3.0,NaN,4.0,...,NaN,NaN,122754.0,360170.0,2015.0,237416.0,6735.88,NaN,NaN,6.037107e+13
1,14366692,-0.1684,2016-01-01,NaN,NaN,NaN,3.5,4.0,NaN,NaN,...,NaN,NaN,346458.0,585529.0,2015.0,239071.0,10153.02,NaN,NaN,NaN
2,12098116,-0.0040,2016-01-01,1.0,NaN,NaN,3.0,2.0,NaN,4.0,...,NaN,NaN,61994.0,119906.0,2015.0,57912.0,11484.48,NaN,NaN,6.037464e+13
3,12643413,0.0218,2016-01-02,1.0,NaN,NaN,2.0,2.0,NaN,4.0,...,NaN,NaN,171518.0,244880.0,2015.0,73362.0,3048.74,NaN,NaN,6.037296e+13
4,14432541,-0.0050,2016-01-02,NaN,NaN,NaN,2.5,4.0,NaN,NaN,...,2.0,NaN,169574.0,434551.0,2015.0,264977.0,5488.96,NaN,NaN,6.059042e+13


In [10]:
train_data_merged.shape

(90275, 60)

Let's put it in a directory called 'interim' so we can read it in in the next notebook.

In [11]:
# write out the training data to csv so we can load it into the next notebook
train_data_merged.to_csv('data/interim/train_data_merged_2016.csv', index=False)